<a href="https://colab.research.google.com/github/swahareddy/gpt2AI_whatsapp_simulation/blob/master/Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: November 10th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [38]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

In [39]:
chat_name="Tripti Rajput"
chat_name_formatted=chat_name.lower().replace(" ","_")

### Format Whatsapp Chat
Convert messages like `13/03/20, 11:34 - Tejas: What's the problem` into `Tejas: What's the problem`



In [40]:
file=open("/content/drive/My Drive/Colab Notebooks/GPT2_WA/Original chats/WhatsApp Chat with "+chat_name+".txt",'r', encoding="utf8")
chat_txt=file.read()
print(chat_txt)
file.close()

import re

lines_match=[]
timestamp_regex="[0-9][0-9]/[0-9][0-9]/[0-9][0-9], [0-9][0-9]:[0-9][0-9] - "
[lines_match.append([m.start(0), m.end(0)]) for m in re.finditer(timestamp_regex, chat_txt)]
# print(lines_match)

from operator import itemgetter
lines_match_sorted=sorted(lines_match, key=itemgetter(0))

chat_wo_ts=""

index_c=0
for name_match in lines_match_sorted:
    if index_c+1<len(lines_match_sorted):
        chat_wo_ts=chat_wo_ts+chat_txt[name_match[1]:lines_match_sorted[index_c+1][0]]
        index_c=index_c+1
    else:
        #print(chat_txt[name_match[1]:])
        chat_wo_ts=chat_wo_ts+chat_txt[name_match[1]:]
        
print(chat_wo_ts)
f = open("/content/drive/My Drive/Colab Notebooks/GPT2_WA/Formatted chats/formatted_"+chat_name_formatted+".txt", "a",  encoding="utf8")
f.write(chat_wo_ts)
f.close()

12/03/20, 19:16 - Tejas: In planning features, architecture etc
12/03/20, 19:17 - Tejas: But since she's in the US she gets to talk to the scrum master and product owner since time zone is the same
12/03/20, 19:17 - Tejas: And I end up playing catch-up when they are sleeping :(
12/03/20, 19:17 - Tejas: Not cool
12/03/20, 19:33 - Tejas: Got shortlisted by FMS Delhi. Will have interview in Delhi sometime in second half of April. However conversion ratio is 1:10. So 🤞🏼
13/03/20, 00:01 - Tripti Rajput: Hi boop
13/03/20, 00:01 - Tripti Rajput: 🥰🥰❤❤❤❤❤❤❤❤❤😋😋
13/03/20, 00:01 - Tejas: It's been 13 hours since we last talked
13/03/20, 00:01 - Tripti Rajput: Fuck wow
13/03/20, 00:01 - Tejas: 4 hours since I'm calling you
13/03/20, 00:01 - Tripti Rajput: Yeas love but if which I was sleeping for more than 10hrs
13/03/20, 00:02 - Tripti Rajput: That's why I am telling you to join  MS in US
13/03/20, 00:54 - Tejas: I was excited that your vacations have started
13/03/20, 00:54 - Tejas: You'll be ab

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [41]:
gpt2.download_gpt2(model_name="774M")

Fetching checkpoint: 1.05Mit [00:00, 441Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 47.8Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 780Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:49, 62.4Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 378Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 157Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 151Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [42]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [43]:
file_name = "formatted_"+chat_name_formatted+".txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [55]:
drive_file_path="Colab Notebooks/GPT2_WA/Formatted chats/"+file_name
import os
home_path="/content"
os.chdir(home_path)
import shutil
shutil.copyfile("/content/drive/My Drive/" + drive_file_path, home_path+"/"+file_name)

'/content/formatted_tripti_rajput.txt'

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()
# import tensorflow
# tensorflow.reset_default_graph()

run_name_var="run_"+chat_name_formatted
print(run_name_var)

gpt2.finetune(sess,
              dataset=file_name,
              model_name='774M',
              steps=750,
              restore_from='fresh',
              run_name=run_name_var,
              print_every=10,
              sample_every=250,
              save_every=250
              )

run_tripti_rajput
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/774M/model.ckpt
INFO:tensorflow:Restoring parameters from models/774M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.22s/it]


dataset has 484127 tokens
Training...


After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name=run_name_var)

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name=run_name_var)

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_name_var)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# drive.flush_and_unmount()

In [ ]:
import os
os.stat('/content/drive/My Drive/Colab Notebooks/GPT2_WA/Chat models/run_triptirajput/model-1000.meta').st_size

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
gpt2.generate(sess, run_name=run_name_var)

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              run_name=run_name_var,
              length=30,
              top_k=40,
              top_p=0.9,
              temperature=1.0,
              # prefix="COEP",
              # include_prefix=False,
              # truncate='COEP',
              nsamples=2,
              batch_size=2
              )

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

`ValueError: Variable model/wpe already exists, disallowed.`

I would try, in this order:

to restart the Google Colab session,
to use a different run_name.

In [ ]:
!kill -9 -1